# **HW2: Decision Tree** 
In *assignment 2*, you need to:

1.   Implement the decision tree in 3 steps (with the example data)


*   Step 1: calculate the entropy
*   Step 2: search for the best split
*   Step 3: build the decision tree

2.   Predict the patients' death (hospDIED) in the *MIMIC* dataset

Please fill in your **studentID** here.

In [1]:
STUDENT_ID = '108062373'

# **1. Implement the Decision Tree**
In the first part, you need to implement the decision tree by completing the given funcitions.

Also, you need to run those functions with given input variables and save the output to the implementation csv file **[STUDENT_ID]_implementation.csv**.

Implement a binary decision tree to classify *Restaurant* dataset.

## Import Packages

Note: You **cannot** import any other packages in the first part (implementation)!

In [2]:
import numpy as np
import pandas as pd
import random

## Load the Example Data
First, load the *Restaurant waiting* dataset: **data.csv**

In [3]:
example = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/data.csv')
example

,Unnamed: 0,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,X1,T,F,F,T,Some,High,F,T,French,8,T
1,X2,T,F,F,T,Full,Low,F,F,Thai,40,F
2,X3,F,T,F,F,Some,Low,F,F,Burger,8,T
3,X4,T,F,T,T,Full,Low,F,F,Thai,12,T
4,X5,T,F,T,F,Full,High,F,T,French,70,F
5,X6,F,T,F,T,Some,Medium,T,T,Italian,3,T
6,X7,F,T,F,F,None,Low,T,F,Burger,7,F
7,X8,F,F,F,T,Some,Medium,T,T,Thai,6,T
8,X9,F,T,T,F,Full,Low,T,F,Burger,80,F
9,X10,T,T,T,T,Full,High,F,T,Italian,20,F


In [4]:
example = example.drop(['Unnamed: 0'], axis=1)

In [5]:
# change the string categorical to integer labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

example['Alternate'] = labelencoder.fit_transform(example['Alternate'])
example['Bar'] = labelencoder.fit_transform(example['Bar'])
example['Friday'] = labelencoder.fit_transform(example['Friday'])
example['Hungry'] = labelencoder.fit_transform(example['Hungry'])
example['Patrons'] = labelencoder.fit_transform(example['Patrons'])
example['Price'] = labelencoder.fit_transform(example['Price'])
example['Raining'] = labelencoder.fit_transform(example['Raining'])
example['Reservation'] = labelencoder.fit_transform(example['Reservation'])
example['Type'] = labelencoder.fit_transform(example['Type'])

example

,Alternate,Bar,Friday,Hungry,Patrons,Price,Raining,Reservation,Type,WaitEstimate,Wait
0,1,0,0,1,2,0,0,1,1,8,T
1,1,0,0,1,0,1,0,0,3,40,F
2,0,1,0,0,2,1,0,0,0,8,T
3,1,0,1,1,0,1,0,0,3,12,T
4,1,0,1,0,0,0,0,1,1,70,F
5,0,1,0,1,2,2,1,1,2,3,T
6,0,1,0,0,1,1,1,0,0,7,F
7,0,0,0,1,2,2,1,1,3,6,T
8,0,1,1,0,0,1,1,0,0,80,F
9,1,1,1,1,0,0,0,1,2,20,F


## Calculating the Entropy *(20%)*


In [6]:
# split the data by given attribute and its threshold
def partition(data, column, threshold):
  """
  The *partition* function will split the input data into 2 branches.
    args:
    *   data(DataFrame): the input data
    *   column(str): the attribute(column name)
    *   threshold(float): the column's threshold for splitting the data
    returns:
    *   match_branch(DataFrame): the divided data that matches the assigned column's threshold
    *   false_branch(DataFrame): the divided data that doesn't match the assigned column's threshold
  """

  match_branch = data[data[column] <= threshold]
  false_branch = data[data[column] > threshold]

  return match_branch, false_branch

If we try *partition(example, 'Patrons', 1.5)*:

split example to 

match_left(Patrons<= 1.5) (Patrons != Some) and  
false_right(Patrons > 1.5) (Patrons == Some)

column: Patrons, threshold: 1.5



In [7]:
#match_left, false_right = partition(example, 'Patrons', 1.5)
match_left, false_right = partition(example, 'Hungry', 0.5)
print(match_left.shape)
print(false_right.shape)
#match_left
#match

(5, 11)
(7, 11)


In [8]:
def entropy(data):
  """
  The *entropy* function will calculate the entropy of the node(data)
  args:
  *    data(DataFrame): the data you're calculating for the entropy
  output:
  *    entropy(float): the node(data)'s entropy     
  """
  Value_Count = data['Wait'].value_counts()
  True_Count = 0
  False_Count = 0

  #print(Value_Count)
  for i, v in Value_Count.items():
    if(i == 'T'):
      True_Count = v
    if(i == 'F'):
      False_Count = v

  entropy = 0
  Prob_True = True_Count / (True_Count + False_Count)
  Prob_False = False_Count / (True_Count + False_Count)

  if(Prob_True > 0 and Prob_False > 0):
    entropy = -( Prob_True * np.log2(Prob_True) + Prob_False * np.log2(Prob_False) )
  #print("Count", True_Count, " ", False_Count)

  return entropy

An example of the output from entropy (Use this to check if your output is correct): 

In [9]:
match, false = partition(example, 'Hungry', 0.5)
entropy1 = entropy(match)
#match
print(entropy1)

0.7219280948873623


In [10]:
#implementation 1: calculate the entropy of 'false_right' data
ans_entropy = entropy(false)
print(ans_entropy)


0.863120568566631


## Find the Best Split *(20%)*

In [11]:
# search for the best attribute and the value(threshold) to split the data
def findBestSplit(data):
    """
    The *findBestSplit* function finds the best combination of attribute and value(with the largest reduction in entropy) to split the data.
    args:
    *   data(DataFrame): the data you try to split(build the decision tree)
    output:
    *   column_best(str): the attribute(column) split with the largest reduction in entropy
    *   value_best(float): the value(threshold) of the column_best attribute to split the data
    """
    all_entropy = 9999        #the overall entropy : save the value of the smallest entropy
    column_best = 0
    value_best = 0
    Choice = {}
    #data_len = len(data.index)
    for col in data:
      if(col == "Wait"):
        continue
      sorted_df = data.sort_values(by=col, ascending=True)
      thresholds = []
      Entropys = []
      #print(sorted_df[col])
      for idx, row in enumerate(sorted_df[col]):
        if(idx == 0):
          continue
        if(sorted_df.iloc[idx][col] != sorted_df.iloc[idx - 1][col]):
          threshold = (float(sorted_df.iloc[idx][col]) + float(sorted_df.iloc[idx - 1][col]))/2
          thresholds.append(threshold)
          #print(sorted_df.iloc[idx][col], ' ', sorted_df.iloc[idx - 1][col], ' ', threshold)

      if(len(thresholds) == 0):
        continue

      #print(thresholds)
      #print(thresholds)
      for threshold in thresholds:
        #print(col)
        left, right = partition(data, col, threshold)
        #print(right)
        left_entropy = entropy(left)
        right_entropy = entropy(right)

        left_count = len(left.index)
        right_count = len(right.index)
        total_count = left_count + right_count

        Entropy = left_entropy * (left_count / total_count) + right_entropy * (right_count / total_count)
        Entropys.append(Entropy)
        #print(Entropy)
        #match_left
      #print(Entropys)
      #print(thresholds)
      min_idx = 0
      min_entropy = Entropys[0]
      for idx, val in enumerate(Entropys):
        if(val < min_entropy):
          min_idx = idx
          min_entropy = val
      Choice[col] = [min_entropy, thresholds[min_idx]]

    #Choice = {k: v for k, v in sorted(Choice.items(), key=lambda item: item[1])}
      #print(col)
    #column_best = list(Choice)[0]
    #value_best = list(Choice.values())[0]
    #print(Choice)

    column_best = next(iter(Choice))
    min_entropy = Choice[column_best][0]
    value_best = Choice[column_best][1]

    #print("1st", min_entropy)

    for key, value in Choice.items():
      if(value[0] < min_entropy):
        min_entropy = value[0]
        value_best = value[1]
        column_best = key


    return column_best, value_best
  

An example of the output from findBestSplit (Use this to check if your output is correct):

In [12]:
column_best, value_best = findBestSplit(match)
print(column_best)
print(value_best)

Patrons
1.5


In [13]:
#implementation 2: Find the best split of the 'match' data
ans_column, ans_value = findBestSplit(false)
#column_best, value_best = findBestSplit(match)
#print(column_best)
#print(value_best)
#match
print(ans_column, ans_value)

WaitEstimate 16.0


## Decision Tree Building *(30%)*

Use the above functions to help building the decision tree





In [14]:
class Node:
  def __init__(self, column, threshold):
    self.left = None
    self.right = None
    self.column = column
    self.threshold = threshold
    #self.isleaf = False
    

def dfs(node):
  if node:
    print(node.column)
    dfs(node.left)
    dfs(node.right)

def is_unique(s):
  a = s.to_numpy()
  return (a[0] == a).all()

def buildTree(df, depth):
  """
  The *buildTree* function builds the decision tree
  args:
  *     df(DataFrame): the data you want to apply the decision tree
  *     depth(int) : the depth of your tree
  output:
  *     decisionSubTree(dict): the decision tree structure including root, branch, leaf(with the attributes and thresholds)
  *     features(list): the features(attributes) name in the decision tree structure(from root to branch and leaf)
  *     thresholds(list): the corresponding thresholds for the features in the 'features' list
  """
  features = []
  thresholds = []
  left_feature = []
  right_feature = []
  left_threshold = []
  right_threshold = []
  decisionSubTree = None

  if(depth > 0):
    column_best, value_best = findBestSplit(df)
    
    decisionSubTree = Node(column_best, value_best)

    left_df = df.loc[df[column_best] <= value_best]
    left_isleaf = is_unique(left_df["Wait"])
    right_df = df.loc[df[column_best] > value_best] 
    right_isleaf = is_unique(right_df["Wait"])

    #right_df

    left_df = df.copy(deep = True)
    right_df = df.copy(deep = True)

    del left_df[column_best]
    del right_df[column_best]
    #print(depth)
    
    if(left_isleaf == False):
      left, left_feature, left_threshold = buildTree(left_df, depth - 1)
      decisionSubTree.left = left

    if(right_isleaf == False):  
      right, right_feature, right_threshold = buildTree(right_df, depth - 1)
      decisionSubTree.right = right

    features = [column_best] + left_feature + right_feature
    thresholds = [value_best] + left_threshold + right_threshold

    #features.append(column_best)
    #thresholds.append(value_best)

  return decisionSubTree, features, thresholds
  

An example of the output from buildTree (Use this to check if your output is correct): 

In [15]:
tree, features, thresholds= buildTree(example, 2)
print(tree)
print(features)
print(thresholds)

['Patrons', 'Hungry']
[1.5, 0.5]


In [16]:
#implementation 3: decision tree building (depth=10)
ans_tree, ans_features, ans_thresholds = buildTree(example,10)
#tree, features, thresholds= buildTree(example, 2)
#example
#print(tree)
#print(features)
#print(thresholds)
#dfs(tree)

In [17]:
#save implementation csv
ans_path = STUDENT_ID + '_implementation.csv'

imp = []
imp.append(ans_entropy)
imp.append(ans_column)
imp.append(ans_value)

for i in range(len(ans_features)):
  imp.append(ans_features[i])
for m in range(len(ans_thresholds)):
  imp.append(ans_thresholds[m])
print(imp)
pd.DataFrame(imp).to_csv(ans_path, header = None, index = None)

[0.863120568566631, 'WaitEstimate', 16.0, 'Patrons', 'Hungry', 'WaitEstimate', 'Price', 'Friday', 'Reservation', 'Alternate', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Alternate', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Reservation', 'Alternate', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Alternate', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Price', 'Friday', 'Reservation', 'Alternate', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Alternate', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Bar', 'Raining', 'Type', 'Type', 'Raining', 'Type', 'Type', 'Reservation', 'Alternate', 

# **2. Classification with the MIMIC Dataset**
In the second part, you need to classify **'hospDIED'(death)** in the MIMIC dataset.

Please put the classification result in a csv file. (**[STUDENTID]_prediction.csv**)

**Note:** Decision tree is recommended but not mandatory.

## Prediction (Performance) *(20%)*

The **y_test** (hospDIED) of this data is hidden, you need to use the x_test.csv to predict the y_test.

**Note:** You **can** now import the packages you need here!

Get the MIMIC data here!

Data Description: 

[Data Description](https://docs.google.com/spreadsheets/d/1pxqxQFhIcv_hrgWEtwhXE6zBVQ5ISa-13PIhvXMtWCY/edit?usp=sharing) (You can find the data description here.)

**Note:**
*   You can select the features you want to use.
*   You can use any ML models to predict the y_test.

  (However, there is a 10% bonus if you **visualize** the decision tree in this part)


[ref]: *Johnson, A. E. W., Pollard, T. J., Shen, L., Lehman, L. H., Feng, M., Ghassemi, M., Moody, B., Szolovits, P., Celi, L. A., & Mark, R. G. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.*


In [25]:
#Read data
x_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/y_train.csv')
#x_train
#x_train.head()
df = pd.concat([x_train, y_train], axis=1, join="inner")
print(len(x_train))

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
x_train['los'] = labelencoder.fit_transform(x_train['los'])
x_train['indextime'] = labelencoder.fit_transform(x_train['indextime'])
df['los'] = labelencoder.fit_transform(df['los'])
df['indextime'] = labelencoder.fit_transform(df['indextime'])

24641


Complete your model with validation:

In [26]:
# change the string categorical to integer labels
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

df['los'] = labelencoder.fit_transform(df['los'])
df['indextime'] = labelencoder.fit_transform(df['indextime'])



#Split the data into training and validation sets
#Build you model and evaluate it on your validation set
#df_train = df.iloc[:18000,:]
#df_validation = df.iloc[18001:,:]
#ans_tree, ans_features, ans_thresholds = MIMIC_buildTree(df_train, 2)

df_x_train = x_train.iloc[:18000,:]
df_x_validation = x_train.iloc[18001:,:]
df_y_train = y_train.iloc[:18000,:]
df_y_validation = y_train.iloc[18001:,:]
#ans_tree, ans_features, ans_thresholds = MIMIC_buildTree(df_train, 2)
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df_x_train, df_y_train)
a = clf.predict(df_x_validation)

TP = 0
FP = 0
FN = 0
TF = 0
for i in range(0, len(a)-1):
  if(a[i] == 1 and df_y_validation.iloc[i]["hospDIED"] == 1):
    TP += 1
  elif(a[i] == 0 and df_y_validation.iloc[i]["hospDIED"] == 0):
    TF += 1
  elif(a[i] == 0 and df_y_validation.iloc[i]["hospDIED"] == 1):
    FN +=1
  else:
    FP += 1
precision = TP/(TP+FP)
recall = TP/(TP+FN)
F1 = 2*(recall * precision)/(recall+precision)
#print(F1)

0.42303664921465967


Make the final prediction from your model:

In [20]:
#x_test = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_test.csv')

#y_pred = 

x_test = pd.read_csv('https://raw.githubusercontent.com/aubreyyy24/HW2_data/main/x_test.csv')
x_test['los'] = labelencoder.fit_transform(x_test['los'])
x_test['indextime'] = labelencoder.fit_transform(x_test['indextime'])

y_pred = clf.predict(x_test)

In [21]:
#final prediction with your decision tree(optional)
def treePredictions(df, tree):

  return predictions

#y_pred = treePredictions(x_test, tree)

To export your predcition as a CSV file and hand in the CSV on elearn

In [22]:
output_path = STUDENT_ID + '_prediction.csv'

tree_test_pred = pd.DataFrame( {'subject_id': x_test.subject_id,
                  'prediction': y_pred } )
tree_test_pred.to_csv(output_path, index = False)

## Visualizing the Decision Tree *(10% bonus)*

**Note:** Save the visualization result image as **[STUDENT_ID]_visualization.png**

*   Your visualization image of the decision tree can contain **five** layers at most.


In [23]:
#Decision Tree Visualization

# Report *(10%)*

Report should be submitted as a pdf file! (**[STUDENT_ID]_report.pdf**)

*   List the top 3 splitting features and their thresholds of your model
*   Briefly describe how you build the decision tree
*   Describe if you apply any improvement on your decision tree model
*   If you preprocess the MIMIC data in the second part (selecting features...), describe the work and reasons
*   Summarize your work
*   Do not exceed 2 pages!






# Save the Code File
Please save your code and submit it as an ipynb file! (**[STUDENT_ID]_hw2.ipynb**)